In [47]:
import os 
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

In [ ]:
import torch
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# Own library imports
from vecopsciml.utils import TensOps
from vecopsciml.operators.zero_order import Mx, My
from vecopsciml.kernels.derivative import DerivativeKernels

# Function from this project
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

# Import model


In [63]:
# Dataset
dataset = 'non_linear'
N_data = 100
noise = 1

data_name = dataset + '_' + str(N_data) + '_' + str(noise)

In [ ]:
# Model
model = 'baseline'
n_modes = 10

model_name = model + '_model_' + str(n_modes)

In [72]:
ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), "../../"))
DATA_PATH = os.path.join(ROOT_PATH, r'data/', data_name, data_name) + '.pkl'
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/', data_name)
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/', data_name, model_name)

In [73]:
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder successfully created at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear_100_1
Folder successfully created at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear_100_1/baseline_model_10


In [66]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear_100_1/non_linear_100_1.pkl


In [67]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [68]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


In [69]:
# Train data splitting in train/test
X = torch.tensor(dataset['X_train'], dtype=torch.float32).unsqueeze(1)
y = torch.tensor(dataset['y_train'], dtype=torch.float32).unsqueeze(1)
K = torch.tensor(dataset['k_train'], dtype=torch.float32).unsqueeze(1)
f = torch.tensor(dataset['f_train'], dtype=torch.float32).unsqueeze(1)

X_train, X_test, y_train, y_test, K_train, K_test, f_train, f_test = train_test_split(X, y, K, f, test_size=0.3, random_state=42)

# Data processing and adequacy with our TensOps library
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)

y_train = TensOps(y_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

# Loading and processing validation data
X_val = torch.tensor(dataset['X_val'], dtype=torch.float32).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

In [ ]:
# Predictive network architecture
input_shape = X_train[0].shape
predictive_layers = [20, 10, n_modes, 10, 20]
predictive_output = y_train.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train)).values[0].shape

# Other parameters
n_filters_explanatory = 5

In [74]:
# Load model and the optimizer
model = PGNNIVBaseline(input_shape, predictive_layers, predictive_output, explanatory_input, explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 100

batch_size = 64
n_checkpoints = 10

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 1.154e+09, Test loss: 1.128e+09, MSE(e): 1.034e+02, MSE(pi1): 1.178e+04, MSE(pi2): 4.512e+01, MSE(pi3): 2.169e+01
Epoch 10, Train loss: 8.055e+07, Test loss: 8.156e+07, MSE(e): 7.832e+00, MSE(pi1): 2.128e+02, MSE(pi2): 4.018e+00, MSE(pi3): 1.025e+00
Epoch 20, Train loss: 6.100e+07, Test loss: 5.552e+07, MSE(e): 5.959e+00, MSE(pi1): 1.355e+02, MSE(pi2): 2.540e+00, MSE(pi3): 5.143e-01
Epoch 30, Train loss: 4.109e+07, Test loss: 5.243e+07, MSE(e): 4.069e+00, MSE(pi1): 3.761e+01, MSE(pi2): 1.929e+00, MSE(pi3): 2.272e-01
Epoch 40, Train loss: 3.282e+07, Test loss: 3.861e+07, MSE(e): 3.278e+00, MSE(pi1): 2.516e+00, MSE(pi2): 1.548e+00, MSE(pi3): 2.124e-01
Epoch 50, Train loss: 3.074e+07, Test loss: 3.673e+07, MSE(e): 3.068e+00, MSE(pi1): 3.462e+00, MSE(pi2): 1.411e+00, MSE(pi3): 2.163e-01
Epoch 60, Train loss: 2.941e+07, Test loss: 3.544e+07, MSE(e): 2.935e+00, MSE(pi1): 3.192e+00, MSE(pi2): 1.366e+00, MSE(pi3): 2.264e-01
Epoch 70, Train l

In [77]:
# Parametros de entrenamiento
start_epoch = 1800
n_epochs = 4000

batch_size = 64
n_checkpoints = 10

second_lr = 1e-4

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 1800.
Epoch 1800, Train loss: 5.911e+06, Test loss: 6.855e+06, MSE(e): 5.862e-01, MSE(pi1): 2.680e+00, MSE(pi2): 3.954e-01, MSE(pi3): 2.153e-01
Epoch 1900, Train loss: 5.821e+06, Test loss: 6.704e+06, MSE(e): 5.772e-01, MSE(pi1): 2.666e+00, MSE(pi2): 3.911e-01, MSE(pi3): 2.153e-01
Epoch 2000, Train loss: 5.740e+06, Test loss: 6.568e+06, MSE(e): 5.692e-01, MSE(pi1): 2.653e+00, MSE(pi2): 3.872e-01, MSE(pi3): 2.152e-01
Epoch 2100, Train loss: 5.667e+06, Test loss: 6.446e+06, MSE(e): 5.618e-01, MSE(pi1): 2.641e+00, MSE(pi2): 3.836e-01, MSE(pi3): 2.151e-01
Epoch 2200, Train loss: 5.599e+06, Test loss: 6.336e+06, MSE(e): 5.551e-01, MSE(pi1): 2.630e+00, MSE(pi2): 3.803e-01, MSE(pi3): 2.151e-01
Epoch 2300, Train loss: 5.537e+06, Test loss: 6.236e+06, MSE(e): 5.489e-01, MSE(pi1): 2.619e+00, MSE(pi2): 3.772e-01, MSE(pi3): 2.150e-01
Epoch 2400, Train loss: 5.480e+06, Test loss: 6.148e+06, MSE(e): 5.432e-01, MSE(pi1): 2.609e+00, MSE(pi2): 3.742e-01, MSE(p